In [21]:
from plasmapy import simulation

from plasmapy.formulary import magnetostatics

import astropy.units as u
import numpy as np
radius = 1 * u.m
main_current = 15 * u.MA
plasma_wire = magnetostatics.CircularWire([0, 0, 1], u.Quantity((0, 0, 0), u.m), radius, main_current)
plasma_wire

CircularWire(normal=[0. 0. 1.], center=[0. 0. 0.], radius=1.0, current=15000000.0)

In [22]:
n_coils = 8
coil_angles = np.linspace(0, 2*np.pi, n_coils, endpoint=False)
coil_angles

array([0.        , 0.78539816, 1.57079633, 2.35619449, 3.14159265,
       3.92699082, 4.71238898, 5.49778714])

In [23]:
minor_radius = 0.3 * u.m
currents = u.Quantity(n_coils * [10e6], u.A)
currents

<Quantity [10000000., 10000000., 10000000., 10000000., 10000000.,
           10000000., 10000000., 10000000.] A>

In [24]:
coils = []
for i in range(n_coils):
    coil_angle = coil_angles[i]
    x = radius * np.cos(coil_angle)
    y = radius * np.sin(coil_angle)
    normal_angle = np.pi/2 + coil_angle
    normal = u.Quantity([np.cos(normal_angle), np.sin(normal_angle), 0])
    center = u.Quantity([x, y, 0 * u.m])
    coil = magnetostatics.CircularWire(normal, center, minor_radius, currents[i])
    coils.append(coil)

In [25]:
all_currents = coils + [plasma_wire]

# TODO Add functions to visualize our MagnetoStatics

Maybe I'm crazy but Mayavi could be a good fit!

# TODO units in reprs for MagnetoStatics

# TODO MagnetoStatics.magnetic_field accepts no units, just numpy arrays!

# TODO mendeleev vs atomic - comparison

In [26]:
plasma_wire.magnetic_field(center.value)

<Quantity [ 0.        ,  0.        , 10.60397635] T>

In [33]:
from plasmapy.classes.plasma_base import GenericPlasma

E_unit = u.V / u.m
class Coils(GenericPlasma):
    """
    Work-in-progress class for passing analytical functions as fields 

    This is primarily helpful for `plasmapy.simulation.ParticleTracker`.
    """

    def __init__(self, *magnetostatics):
        """
        Initialize plasma paramters.
        The most basic description is composition (ion), temperature,
        density, and ionization.
        """
        self.magnetostatics = magnetostatics
    
    def interpolate_E(self, r: u.m):
        return u.Quantity(np.zeros(r.shape), E_unit)
    
    def _interpolate_B(self, r):
        B = np.zeros(r.shape)
        for ms in self.magnetostatics:
            field = ms._magnetic_field(r[0], ms.pt, ms.dl, ms.current)
            B[0] += field
        return B

    def interpolate_B(self, r: u.m):
        return u.Quantity(self._interpolate_B(r.si.value), u.T)

    @classmethod
    def is_datasource_for(cls, **kwargs):
        match = 'interpolate_B' in kwargs.keys()
        return match
    
c = Coils(*all_currents)

sim = simulation.ParticleTracker(c, 'e', dt=1e-8 * u.s, nt=int(1e6))
sim._x[0][0] = 1 + minor_radius.si.value / 2 # * (u.m / u.s)
sim._v[0][1] = 10000 # * (u.m / u.s)
sim._v[0][2] = 100 # * (u.m / u.s)


sim.run() # this should return a Solution object or sth

In [34]:
import mayavi
from mayavi import mlab
mlab.init_notebook()

Notebook initialized with ipy backend.


In [35]:
fig = mlab.figure()
for ms in all_currents:
    x, y, z = ms.curve(np.linspace(0, 2*np.pi))
    mlab.plot3d(x, y, z, figure=fig)
x, y, z = sim.position_history[:,0,:].T

trajectory = mlab.plot3d(x,y,z, sim.t, figure=fig, line_width=1e-4, representation='surface')
mlab.colorbar(trajectory, title="Time")
mlab.orientation_axes(figure=fig)
display(fig)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [36]:
sim2 = simulation.ParticleTracker(c, 'e', dt=1e-8 * u.s, nt=int(1e6))
sim2._x[0][0] = 1 + minor_radius.si.value / 2 # * (u.m / u.s)
sim2._v[0][0] = 0 # * (u.m / u.s)
sim2._v[0][1] = 10000 # * (u.m / u.s)
sim2._v[0][2] = 0 # * (u.m / u.s)
sim2.run() # this should return a Solution object or sth

In [38]:
fig2 = mlab.figure()
for ms in all_currents:
    x, y, z = ms.curve(np.linspace(0, 2*np.pi))
    mlab.plot3d(x, y, z, figure=fig2)
x, y, z = sim2.position_history[:,0,:].T

trajectory = mlab.plot3d(x,y,z, sim2.t, figure=fig2, line_width=1e-13, representation='surface')
mlab.colorbar(trajectory, title="Time")
mlab.orientation_axes(figure=fig2)
display(fig2)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [68]:
factor = 1.7
x = np.linspace(-factor * (radius + minor_radius), factor * (radius + minor_radius), 100)
z = np.linspace(-2*factor * minor_radius, 2*factor * minor_radius, 100)
X, Y, Z = np.meshgrid(x, x, z, indexing='ij')

Bval = np.zeros((3, *Z.shape))

from tqdm import auto as tqdm

for i, xi in tqdm.tqdm(enumerate(x.si.value), total=len(x)):
    for j, yi in enumerate(x.si.value):
        for k, zi in enumerate(z.si.value):
            pos = np.array([[xi, yi, zi]])
            field = c._interpolate_B(pos)
            Bval[:, i,j,k] = field

Bmag2 = np.sum(Bval**2, axis=0)

In [70]:
fig = mlab.figure(size=(800, 600))
for ms in all_currents:
    x, y, z = ms.curve(np.linspace(0, 2*np.pi))
    mlab.plot3d(x, y, z, figure=fig)

contours = mlab.contour3d(X.value, Y.value, Z.value, np.log10(Bmag2), figure=fig,
#                           contours = np.linspace(Bmag2.min(), Bmag2.max(), 50).tolist(),
#                           contours = np.logspace(np.log10(Bmag2.min()),
#                                                  np.log10(Bmag2.max()),
#                                                  num=10,
#                                                 ).tolist(),
                          contours = 15,
                          opacity = 0.3,
                         )
mlab.colorbar(contours, title="log10 |B|^2")
mlab.orientation_axes(figure=fig)
x, y, z = sim.position_history[:,0,:].T

trajectory = mlab.plot3d(x,y,z, sim.t, figure=fig, line_width=1e-4, representation='surface')
mlab.colorbar(trajectory, title="Trajectory - time", orientation='vertical')
display(fig)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03 \x00\x00\x02X\x08\x02\x00\x00\x00\x15\x14\x15\'\…